###Import necessary libraries and convert csv to dataframe

In [57]:
#import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#convert the csv file to a dataframe
fuel_df = pd.read_csv('/content/Fuel.csv')
#to check if we have successfully obtained a dataframe, we can print the first 5 rows of the dataframe using the head() function
#fuel_df.head()

###Check for NaN values

In [58]:
'''
the isnull() function can be used to check where all there is a NaN value in the dataframe
and then we can run a for loop to check whether a given column has NaN values and how many (over here knowing the count of NaN values won't be of help but no harm in knowing)
and we can append the count of NaN values in a list
'''
NaN_value_count = []                    #creating an empty list
for column in fuel_df.columns:          #for loop to iterate over each column
  x = fuel_df[column].isnull().sum()    #assigning the number of NaN values in each column to x
  NaN_value_count.append(x)             #append x into a list (NaN_value_count)
print(NaN_value_count)
#[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] this is the output that we get - so there are no NaN values

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### Checking which features are important
*   We will keep all the numeric features
*   We remove MODELYEAR as all values are 2014. So removing it won't make any difference

In [59]:
#since all the values are 2014, hence we can remove MODELYEAR
fuel_df.drop('MODELYEAR', axis = 1, inplace = True)

###Encoding categorical columns and normalizing the data using mean normalization

In [60]:
#encoding the categorical columns
fuel_df['MAKE'] = fuel_df.MAKE.astype('category').cat.codes                   #MAKE
fuel_df['MODEL'] = fuel_df.MODEL.astype('category').cat.codes                 #MODEL
fuel_df['VEHICLECLASS'] = fuel_df.VEHICLECLASS.astype('category').cat.codes   #VEHICLECLASS
fuel_df['TRANSMISSION'] = fuel_df.TRANSMISSION.astype('category').cat.codes   #TRANSMISSION
fuel_df['FUELTYPE'] = fuel_df.FUELTYPE.astype('category').cat.codes           #FUELTYPE

#we dont want to normalize the CO2EMISSIONS column hence first remove it
fuel_co2emissions = fuel_df.pop('CO2EMISSIONS')

#now we normalize the data using mean normalization method
#(value - mean)/(standard deviation)
for column in fuel_df.columns:
  fuel_df[column] = (fuel_df[column] - fuel_df[column].mean())/fuel_df[column].std()

#now we will concatenate the CO2EMISSIONS column and the normalized dataframe
fuel_df = pd.concat([fuel_df, fuel_co2emissions], axis = 1)   #axis = 1 because we want the dataframes side by side

#fuel_df

###I will be using both, the normal equation method and gradient descent to obtain predictions. But the normal equation method is better because of the following reasons
*   Number of features is very less
*   No need to choose learning rate
*   Don't need to solve iteratively using gradient descent



#1) Normal Equation Method
###Now how does it work?
The hypothesis function in general is of the form $h_{\theta} = \theta_{0}x_{0} + \theta_{1}x_{1} + \theta_{2}x_{2} + .... + \theta_{n}x_{n}$ where $x_{i}$ represents the feature values of each row and $\theta_{i}$ are the parameters corresponding to each feature. So we basically have to find the values of $\theta_{i}$ (which I have done using the normal equation method) and then multiply these with the data that is there in y_dev
 to get the predictions

In [61]:
#first we create a column of ones (which corresponds to the x0 of the hypothesis function)
fuel_df['x0'] = 1      #lets call the column 'x0'
#this will be added at the end
#as per the hypothesis function, x0 has to be the first column so we rearrange the columns
fuel_df = fuel_df.iloc[:,[12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]

###Splitting the dataset into training and development datasets

In [62]:
#now we split the data into training and development as per a ratio (to determine the number of rows that will be in the training set and the rest in the development set)
ratio = 0.7

#we do this by creating a list of 1067*ratio randomly selected numbers from a range of 0 to 1067 and assign it to the list rand_list
#the elements of rand_list essentially now correspond to the index of a row in fuel_df
rand_list = np.random.choice(np.arange(1067), size=int(1067*ratio), replace=False)

#next we create x_train. we take each element or rand_list and fill it into x_train
x_train = fuel_df.iloc[rand_list]

#now we have to fill the remaining elements into x_dev
#to do this, we create remaining_list, which contains all those values in the range of 0 to 1067 which are not there in rand_list
#this can be done using the setdiff1d() function
remaining_list = np.setdiff1d(np.arange(1067), rand_list)

#next we create x_dev. we take each element of rand_list and fill it into x_dev
x_dev = fuel_df.iloc[remaining_list]

#now we create a seperate dataframe for the target variable (CO2EMISSIONS)
#just remove the CO2EMISSIONS columns from x_train and x_dev and call it y_train and y_dev
y_train = x_train.pop('CO2EMISSIONS')
y_dev = x_dev.pop('CO2EMISSIONS')

To solve for the $\theta_{i}$ values we use the equation $\theta = (X^{T}X)^{-1}X^{T}Y$ where $X$ is the matrix of the values present in x_train, $X^{T}$ is the transpose of $X$, and $Y$ is the values of y_train

In [63]:
#we have to create the required matrices from the dataframes that we have. This can be done using the .copy() function
X = x_train.copy()                         # X
XT = X.T                                   # X^T (X transpose)
Y = y_train.copy()                         # Y
theta_normal_equation = np.linalg.inv(XT @ X) @ XT @ Y     # Theta
#now we have the values of the parameters theta0, theta1, ...., theta11

###Make the Predictions

In [64]:
#we can make the predictions by taking the dot product of x_dev and theta
predictions_normal_equation = np.dot(x_dev, theta_normal_equation)

###Using R-squared to measure the goodness of the fit
##$R-squared = 1 - \frac{TSS}{RSS}$
where TSS is Total sum of squares and RSS is Residual sum of squares

In [65]:
#TSS is the sum of squares of the differences of the values in y_dev from the mean of y_dev
TSS = ((y_dev - y_dev.mean())**2).sum()
#TSS is the sum of squares of the differences of the values in y_dev and predictions
RSS = ((y_dev - predictions_normal_equation)**2).sum()
#calculate r_squared
r_squared_normal_equation = 1 - (RSS/TSS)
print(r_squared_normal_equation)

0.9078999605079476


#2) Gradient Descent Method
The gradient descent algorithm is of the form  

Repeat until convergence {
  
  $\theta_{j} := \theta_{j} - \alpha * 1/m * \displaystyle\sum_{i=1}^{m} (h_{\theta}(x^{(i)}) - y^{(i)})x_{j}^{(i)}$
  
  } Simultaneously update $\theta_{j}$ for j = 0, 1, ..., n

  n = number of features

  m = number of rows of data

  $\alpha$ = learning rate

  $h_{\theta}$ = hypothesis function

In [66]:
#assigning values to hyperparameters
number_of_iterations = 900       #number_of_iterations
alpha = 0.06                     #alpha

#m is the length of y_train
m = len(y_train)
#initialize theta0, theta1, ...., theta11 to zero (can take any value. taking zero for simplicity)
theta_gradient_descent = np.zeros(x_train.shape[1])

#now we loop over the gradient descent algorithm "number_of_iterations" times
for i in range (number_of_iterations):
  theta_gradient_descent = theta_gradient_descent - alpha * (1/m) * XT @ (X @ theta_gradient_descent - y_train)

#we can make the predictions by taking the dot product of x_dev and theta
predictions_gradient_descent = np.dot(x_dev, theta_gradient_descent)

#now we calculate r-squared for gradient descent
TSS = ((y_dev - y_dev.mean())**2).sum()
RSS = ((y_dev - predictions_gradient_descent)**2).sum()
r_squared_gradient_descent = 1 - (RSS/TSS)
print(r_squared_gradient_descent)

0.9086945003761459


###Comparing with sklearn

In [67]:
#import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

#make the model
model = LinearRegression()

#train the model
model.fit(x_train, y_train)

#make the predictions
predictions_skl = model.predict(x_dev)

#measure the goodness of the fit
r_squared_sklearn = r2_score(y_dev, predictions_skl)
print(r_squared_sklearn)

0.9078999605079021


In [68]:
print(f"R-squared: Normal Equation = {r_squared_normal_equation}")
print(f"R-squared: Gradient Descent = {r_squared_gradient_descent}")
print(f"R-squared: Sklearn = {r_squared_sklearn}")

R-squared: Normal Equation = 0.9078999605079476
R-squared: Gradient Descent = 0.9086945003761459
R-squared: Sklearn = 0.9078999605079021
